### Notebook for initializing polymer systems using a DPD potential

In [1]:
import matplotlib
import numpy as np  
import gsd, gsd.hoomd 
import hoomd 
import time
import matplotlib_inline
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use("ggplot")
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

In [19]:
def initialize_snapshot_rand_walk(num_pol, num_mon, density=0.85, bond_length=1.0, buffer=0.1):
    '''
    Create a HOOMD snapshot of a cubic box with the number density given by input parameters.
    '''    
    N = num_pol * num_mon
    L = np.cbrt(N / density)  # Calculate box size based on density
    positions = np.zeros((N, 3))
    for i in range(num_pol):
        start = i * num_mon
        positions[start] = np.random.uniform(low=(-L/2),high=(L/2),size=3)
        for j in range(num_mon - 1):
            delta = np.random.uniform(low=(-bond_length/2),high=(bond_length/2),size=3)
            delta /= np.linalg.norm(delta)*bond_length
            positions[start+j+1] = positions[start+j] + delta
    positions = pbc(positions,[L,L,L])
    bonds = []
    for i in range(num_pol):
        start = i * num_mon
        for j in range(num_mon - 1):
            bonds.append([start + j, start + j + 1])
    bonds = np.array(bonds)
    frame = gsd.hoomd.Frame()
    frame.particles.types = ['A']
    frame.particles.N = N
    frame.particles.position = positions
    frame.bonds.N = len(bonds)
    frame.bonds.group = bonds
    frame.bonds.types = ['b']
    frame.configuration.box = [L, L, L, 0, 0, 0]
    return frame

def pbc(d,box):
    for i in range(3):
        a = d[:,i]
        pos_max = np.max(a)
        pos_min = np.min(a)
        while pos_max > box[i]/2 or pos_min < -box[i]/2:
            a[a < -box[i]/2] += box[i]
            a[a >  box[i]/2] -= box[i]
            pos_max = np.max(a)
            pos_min = np.min(a)
    return d

def check_bond_length_equilibration(snap,num_mon,num_pol,max_bond_length=1.1,min_bond_length=0.9): #todo
    frame_ds = []
    for j in range(num_pol):
        idx = j*num_mon
        d1 = snap.particles.position[idx:idx+num_mon-1] - snap.particles.position[idx+1:idx+num_mon]
        bond_l = np.linalg.norm(pbc(d1,snap.configuration.box),axis=1)
        frame_ds.append(bond_l)
    max_frame_bond_l = np.max(np.array(frame_ds))
    min_frame_bond_l = np.min(np.array(frame_ds))
    print("max: ",max_frame_bond_l," min: ",min_frame_bond_l)
    if max_frame_bond_l <= max_bond_length and min_frame_bond_l >= min_bond_length:
        print("DPD Simulation Finished.")
        return True
    if max_frame_bond_l > max_bond_length or min_frame_bond_l < min_bond_length:
        return False

def run_one(A=1000,gamma=1000,k=1000,num_pol=100,num_mon=10,kT=1.0,r_cut = 1.15,bond_l=1.0,dt=0.001,density=0.8,particle_spacing = 1.1
):
    print(num_pol*num_mon)
    print(f"\nRunning with A={A}, gamma={gamma}, k={k}, "
          f"num_pol={num_pol}, num_mon={num_mon}")
    start_time = time.perf_counter()
    frame = initialize_snapshot_rand_walk(num_pol, num_mon, density=density)
    build_stop = time.perf_counter()
    print("Total build time: ", build_stop-start_time)
    harmonic = hoomd.md.bond.Harmonic()
    harmonic.params["b"] = dict(r0=bond_l, k=k)
    integrator = hoomd.md.Integrator(dt=dt)
    integrator.forces.append(harmonic)
    simulation = hoomd.Simulation(device=hoomd.device.auto_select(), seed=np.random.randint(65535))# TODO seed
    simulation.operations.integrator = integrator 
    simulation.create_state_from_snapshot(frame)
    const_vol = hoomd.md.methods.ConstantVolume(filter=hoomd.filter.All())
    integrator.methods.append(const_vol)
    nlist = hoomd.md.nlist.Cell(buffer=0.4)
    simulation.operations.nlist = nlist
    DPD = hoomd.md.pair.DPD(nlist, default_r_cut=r_cut, kT=kT)
    DPD.params[('A', 'A')] = dict(A=A, gamma=gamma)
    integrator.forces.append(DPD)
    
    simulation.run(0)
    simulation.run(1000)
    snap=simulation.state.get_snapshot()
    N = num_pol*num_mon
    time_factor = N/90000
    
    while not check_bond_length_equilibration(snap,num_mon, num_pol,max_bond_length=particle_spacing): #TODO - work with snapshot instead of gsd?
        check_time = time.perf_counter()
        if (check_time-start_time) > 60*time_factor:
            return num_pol*num_mon, 0
        simulation.run(1000)
        snap=simulation.state.get_snapshot()

    while not check_inter_particle_distance(snap,minimum_distance=0.95):
        check_time = time.perf_counter()
        if (check_time-start_time) > 60*time_factor:
            return num_pol*num_mon, 0
        simulation.run(1000)
        snap=simulation.state.get_snapshot()
        
    end_time = time.perf_counter()
    return num_pol*num_mon, end_time - start_time
    
N,s = run_one(A=1000,gamma=800,k=20000,num_pol=1,num_mon=5000) #A=1000 seems fast, cranking up gamma to 800 speeds up, cranking up k more does too,
#no difference between k=10k and 20k
#OK, with these parameters, we're finishing in under 100 steps
print(f"Finished in time = {s:.2f}s")

5000

Running with A=1000, gamma=800, k=20000, num_pol=1, num_mon=5000
Total build time:  0.02890995901543647
max:  1.0278795950112403  min:  0.9699675961085741
Bonds relaxed.
Inter-particle separation reached.
Finished in time = 1.12s


In [24]:
N,s = run_one(A=1000,gamma=800,k=20000,num_pol=10,num_mon=10000)
print(f"Finished in time = {s:.2f}s")

100000

Running with A=1000, gamma=800, k=20000, num_pol=10, num_mon=10000
Total build time:  0.3765587500529364
max:  1.0357962891142412  min:  0.9641241040101506
Bonds relaxed.
Finished in time = 0.00s


In [23]:
N,s = run_one(A=1000,gamma=800,k=20000,num_pol=100,num_mon=100)
print(f"Finished in time = {s:.2f}s")

10000

Running with A=1000, gamma=800, k=20000, num_pol=100, num_mon=100
Total build time:  0.06813287490513176
max:  1.0307020874075663  min:  0.9736944719590982
Bonds relaxed.
Inter-particle separation reached.
Finished in time = 3.07s


In [17]:
N,s = run_one(A=10000,gamma=800,k=20000,num_pol=200,num_mon=100)
print(f"Finished in time = {s:.2f}s")

20000

Running with A=10000, gamma=800, k=20000, num_pol=200, num_mon=100
Total build time:  0.10833537497092038
max:  1.1233501358960543  min:  0.8812512536892857
max:  1.087185350886018  min:  0.9094451522526453
max:  1.0670725042459877  min:  0.930209991289446
max:  1.0633493527100006  min:  0.9274543224424658
max:  1.0557595494007133  min:  0.9392829253095489
max:  1.0593671450516882  min:  0.9436121150546137
max:  1.0481534408608102  min:  0.9456298981203638
max:  1.0486255086337204  min:  0.9435113052883237
max:  1.047845909265966  min:  0.9504334235611499
Bonds relaxed.
Inter-particle separation reached.
Finished in time = 17.12s
